{[Click aquí para ver este documento en Google Colab](https://colab.research.google.com/drive/1yiet2U7nwC7ufPJJtxBStyvFvnzJEyXV)}

<head><link rel = "stylesheet" href = "https://drive.google.com/uc?id=1zYOH-_Mb9jOjRbQmghdhsmZ2g6xAwakk"></head>

<table class = "header" width = 100%><tr>
    <th align = "left">The S<code>AI</code>X Guys | AI Trading, Octubre 2020</th>
    <th align = "right">Sección: ZMQ</th>
</tr></table>

## <center><b><u>Clase "ZMQL"</u></b></center>

Dentro de nuestro [modelo comunicacional](https://en.wikipedia.org/wiki/Client%E2%80%93server_model), vamos a crear un primer objeto base para que nuestro **cliente en Python** pueda compartir información con nuestro Expert Advisor programado en MQL: "**ZMQ + MQL = ZMQL**". La idea es que cada uno de los bloques que va a interactuar de manera directa con nuestro **servidor en MetaTrader**, sea una subclase de "``ZMQL``", con sus respectivas adaptaciones necesarias. Pero por ahora, vamos a trabajar sobre las herramientas de bajo nivel, comunes a todas ellas.

Dentro del protocolo de **ZeroMQ**, entre los varios tipos de canales de comunicación que existen, vamos a usar los siguientes 3:

|  | &emsp; <u>Emisor</u> &emsp; | &emsp; <u>Receptor</u> &emsp; | <u>Acople</u> | <u>Sentido</u> | &emsp; <u>Filas de espera</u> &emsp; | &emsp; <u>Flujo de datos</u> &emsp; |
|--|:-----------:  |:--------------:|:------------------------------------:|:-----------------------------------:|
| [REQ-REP](https://https://en.wikipedia.org/wiki/Request%E2%80%93response) | Ambos | Ambos | &emsp; Sincrónico &emsp; | &emsp; Bidireccional &emsp; | &emsp;&emsp;&emsp; No | &emsp; Intermitente &emsp; |
| [PAIR-PAIR](https://en.wikipedia.org/wiki/Point-to-point_(telecommunications)) | Ambos | Ambos | &emsp; Asincrónico &emsp; | &emsp; Bidireccional &emsp; | &emsp;&emsp;&emsp; No | &emsp; Intermitente &emsp; |
| [PUSH-PULL](https://en.wikipedia.org/wiki/Push_technology) | **PUSH** | **PULL** | &emsp; Asincrónico &emsp; | &emsp; Unidireccional &emsp; | &emsp;&emsp;&emsp; [Si](https://en.wikipedia.org/wiki/FIFO_(computing_and_electronics)) | &emsp; Intermitente &emsp; |
| [PUB-SUB](https://en.wikipedia.org/wiki/Publish%E2%80%93subscribe_pattern)  | **PUB** | **SUB** | &emsp; Asincrónico &emsp; | &emsp; Unidireccional &emsp; | &emsp;&emsp;&emsp; No | &emsp; Continuo &emsp; |

Para mas información, visitar la [documentación de **ZeroMQ**](https://zguide.zeromq.org/docs/chapter2/).

En primera instancia, comenzamos haciendo un "``import``" de las librerías mas importantes que vamos a usar. Ante la falta de alguna, asegurarse de instalarla en alguna celda o terminal aparte, usando "``pip install``". Estas son:
* "``os``": Herramientas del sistema operativo. Mas que nada, para poder ubicar carpetas y directorios que vayamos a usar.
* "``zmq``": Adaptación oficial del protocolo "**ZeroMQ**" para Python. Las hay para [varios lenguajes](http://wiki.zeromq.org/bindings:_start), uno de ellos siendo MQL4.
* "``time``": Funciones de cronometro. La usamos mas que nada para generar pausas/esperas con la función "``time.sleep``".
* "``datetime``": Funciones de calendario. Fechas, horarios, zonas horarias e intervalos de tiempo.
* "``pandas``": Las DataFrames no solo pueden contener datos alfanuméricos, sino también objetos cualesquiera.
* "``threading``": Para llevar a cabo las tareas de recepción de mensajes detrás del hilo principal, y no interferir con él.
* "``IPython``": Prints/displays de DataFrames con HTML y CSS, con la función "``IPython.display.display``".

In [1]:
import os, sys, zmq, time, datetime, pandas, threading, IPython
ZMQ = zmq.Context()

Como primer paso, debemos crear un objeto "``Context``" al cual llamaremos simplemente "``ZMQ``". Su función es la de reconocer todos los futuros canales de comunicación ZMQ, para que formen parte del mismo protocolo. Vendría a ser como un "espacio en común" que enlaza a todos los clientes y servidores.

### <center><b><u>Inicialización</u></b></center>

<u>**Canales y números de "port"**</u>: Para crear nuestros canales de comunicación, vamos a usar directamente la red Ethernet, puesto que son aplicaciones locales las que utilizaremos. Necesitamos entonces asignarle una dirección "TCP" a cada uno. Para diferenciar un canal de otro, los identificamos con un número de "``port``". Por ejemplo, para un canal con "``port = 777``", la dirección será "``tcp://localhost:777``". Como nuestros canales estan ubicados dentro del ``Context``, necesitamos proveerle este a nuestro objeto **ZMQL**. 

<u>**Sockets**</u>: Un nodo dentro de una red ZeroMQ accede los canales mediante "``Socket``s": portales por medio del cual uno formula y envía, o recibe y procesa mensajes. El ``Socket`` va a conocer el idioma y el conjunto de reglas comunicacionales por medio del cual se (de)codifica/(des)empaqueta la información. Para crear un socket, necesito en primera instancia conectarlo a la dirección TCP del canal, con la función "``.connect``".

Al crear una instancia de ``ZMQL``, como argumento ("``ports``") vamos a usar un ``dict`` adonde las ``keys`` van a ser los nombres que nosotros les ponemos a cada uno de los ``Socket``s, y los valores van a ser los números de ``port``.

Ahora bien: un nodo puede tener varios ``Socket``s, especialmente si la comunicación es bidireccional. Dentro del entorno ZMQ, se distingue uno de otro con un "``Enum``"; un número ("``int``") identificador del tipo de socket. La librería de "``zmq``" ya incluye dichas identidades. Ej: "``zmq.PUSH = 8``", "``zmq.PULL = 7``", "``zmq.SUB = 3``", etc.

Al mismo tiempo, los ``Socket``s pueden llegar a tener que ocupar una cantidad de memoria variable, dependiendo de que tipo sean. Si son "**PUSH** - **PULL**", como los mensajes van a ponerse en fila, necesitamos limitar el tamaño de esa fila, para no saturar nuestro ``Context``. Hacemos esto aplicando un "[watermark](http://api.zeromq.org/2-1:zmq-setsockopt)" en cada ``Socket`` con la función "``setsockopt``".

<u>**DataFrame "``Comm``"**</u>: Todas estas cosas mencionadas, van a estar almacenadas dentro de un atributo "``Comm``". El motivo por el cual usamos un ``DataFrame`` para él, es porque se pueden guardar también objetos de cualquier clase dentro de ellos, y en este caso, sucede que tenemos un set "``Socket`` - ``Port`` - ``Enum``" para conectar un nodo a cada canal. De hecho, como complemento, vamos a añadir una celda de "``Cache``" para cada uno, para siempre almacenar el último mensaje que se envió o recibió (dependiendo del caso), a modo de "inbox".

<u>**Poller**</u>: Otro objeto que será fundamental, ya que nos permite identificar facilmente cuando es que un mensaje llegó a alguno de los ``Socket``s cuyo ``Role`` es de ``R``ecepción (**REQ / PULL / SUB / PAIR**) sin tener que leer lo que hay dentro de ellos de manera directa.

<u>**Variables de control**</u>: Dentro de un ``dict`` llamado "``Enable``", vamos a añadir dos elementos fundamentales, con la idea de ser controlados directamente por el usuario: "``comm``" y "``verbose``". Este último será otro argumento del constructor de **ZMQL**.

Básicamente, "``comm``" activa (``True``) o desactiva (``False``) la lectura de los mensajes recibidos. Sirve para pausar la comunicación sin desenlazar los ``Socket``s, ya sea como forma de corregir algún error (debugging), o para provocar una comunicación unidireccionar (open loop). Se verá su utilidad mas adelante. La variable "``verbose``" es lo que su nombre indica: sirve para habilitar el reporte del sistema frente a los distintos eventos que van sucediendo. Se verá mas en detalle posteriormente, pero debe saberse que incluirá "3 grados posibles":
* "``verbose = 0``": mostrar solo errores o información prioritaria.
* "``verbose = 1``": lo anterior, mas mensajes recibidos salvo **SUB**.
* "``verbose = 2``": todo lo anterior, pero agregando mensajes **SUB**.

In [2]:
class ZMQL(object):
     #| Ubicación de la "Common Data Folder". Todo archivo (CSVs) solicitado a MQL, irá a parar allí.
    _CommonPath = os.path.expanduser("~") + "\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common\\Files\\"
    #| Nombres de puertos; deben llevar al principio y en mayusculas, el TIPO (ej.: "PUSH Data", "REP Trading", etc.)
    _PortsDef = {"SUB": 65530, "PUSH": 65531, "PULL": 65532} #| Puertos de cada socket. Deben ser ints.
    #| Lista de códigos de error de MT4 con sus descripciones, en formato "pandas.Series". Descargar de Google Drive.
    _MQErrors = pandas.read_csv("https://drive.google.com/uc?id=1YFLpNNbJMd-NaZNjEklN4snBD-wcKc9r").set_index("#")
    def __init__(self, context, ports = _PortsDef, verbose = 1):
        assert verbose in (0, 1, 2), "((INIT)) ERROR! \"verbose\" may either be integer \"0\", \"1\" or \"2\"."
        assert isinstance(context, zmq.sugar.context.Context), "((INIT)) ERROR! Use a valid (zmq.) \"context\" input."
        warn = "((INIT)) ERROR! \"ports\" dict may be something like: {\"PUSH ...\": (int), \"PULL ...\": (int), ...}."
        assert isinstance(ports, dict) and all([isinstance(port, int) for port in ports.values()]), warn
        assert (len(ports.values()) == len(set(ports.values()))), warn + " Each key/value must be unique."
        #| Dataframe con objetos de comunicación: con todas las variables de acceso al protocolo.
        self.Comm = pandas.DataFrame(columns = ["Port"], index = ports.keys(), data = ports.values())
        self.Enable = {"comm": True, "verbose": verbose} #| Diccionario con parámetros de control.
        self.Poller = zmq.Poller() #| Inicializar "poll" para empezar a detectar llegada de mensajes.
        print("---------------------------------------"*2)
        for label in self.Comm.index: #| Ir creando cada uno de los puertos de comunicación con MetaTrader.
            assert isinstance(ports[label], int), "((INIT)) ERROR! Ports must be integers and respond to MT4."
            if (verbose > 0): print(f"[[INIT]] {label} connecting to port {ports[label]}. Await for response...")
            enum = eval("zmq." + label.split(' ')[0]) #| "Enum" de ZMQ que distingue que tipo de puerto es.
            self.Comm.loc[label, "Enum"] = enum #| Guardar este "Enum" en el DataFrame de objetos de comunicación.
            if enum in (zmq.PUB, zmq.PUSH, zmq.ROUTER): self.Comm.loc[label, "Role"] = "S" #| Es un "Sender".
            if enum in (zmq.SUB, zmq.PULL, zmq.DEALER): self.Comm.loc[label, "Role"] = "R" #| Es un "Receiver".
            if enum in (zmq.REQ, zmq.REP, zmq.PAIR): self.Comm.loc[label, "Role"] = "SR" #| Alterna entre ambos.
            Socket = context.socket(enum) #| Crear socket: portal de envío, recepción e interpretación de mensajes.
            self.Comm.loc[label, "Socket"] = Socket #| Guardar socket en DataFrame de objetos de comunicación.
            Socket.connect(f"tcp://localhost:{ports[label]}") #| Conectar cada dirección al protocolo común (ethernet).
            if (enum == zmq.SUB): Socket.setsockopt(zmq.SUBSCRIBE, b"{") #| Limitar memoria para cache de puertos SUB.
            Socket.setsockopt(zmq.LINGER, 0) #| Al eliminar los sockets, se elimina cualquier fila de espera que tenga.
            self.Comm.loc[label, "Cache"] = "" #| Crear columna en DataFrame como "inbox de últimos mensajes".
            if "S" in self.Comm.loc[label, "Role"]: Socket.setsockopt(zmq.SNDHWM, 1)
            if "R" in self.Comm.loc[label, "Role"]: Socket.setsockopt(zmq.RCVHWM, 1)
            if "R" in self.Comm.loc[label, "Role"]: self.Poller.register(Socket, zmq.POLLIN)
            #| Initialize poll set for message parsing.
        print("---------------------------------------"*2)
        self.Comm[["Port", "Enum"]] = self.Comm[["Port", "Enum"]].astype(int) #| Guardar "Enums" de ZMQ como ints.

### <center><b><u>Deinicialización</u></b></center>

Es importante tener una forma de borrar todo rastro de la presencia del nodo dentro de la red de canales, una vez que este se elimina. Sucede que no alcanza solamente con hacer un restart del kernel de Python. Aunque los ``Socket``s desaparezcan, los canales siguen inscriptos al ``Context``. Al quitarles un nodo a estos, su flujo de mensajes puede quedar "colgado". Las razones de una función "``shutdown``" son varias:
* Para permitir que otro nodo pueda requerir de ocupar su lugar.
* Para eliminar cualquier rastro de mensajes en una fila que ocupa memoria innecesaria.
* Para que el servidor no pierda tiempo en enviarle mensajes a un nodo que ya no existe.
* Para poder hacer un reset del sistema, sin interferencias, cuando sea necesario.

<u><b>Función "``shutdown``"</b></u>: Fundamentalmente, la idea es que ``ZMQL`` se des-registre del ``Context`` y anule toda tarea pendiente de recepción haciendo ``False`` a la variable de control "``comm``" dentro de "``Enable``". Aparte debe desconectar cada uno de los ``Socket``s dentro de nuestro panel ``Comm`` respecto de su dirección TCP correspondiente. Para los ``Socket``s receptores, se eliminan los listeners del "``Poller``".

Sin embargo, es <u>**muy importante**</u> que antes de eso, se le <u>notifique a los otros nodos de la red acerca de su abandono</u>. ¿<u>**Por qué**</u>?

Los sockets ``PUB`` y ``PUSH``  tienen la capacidad de enviar datos a una gran cantidad de ``SUB``s y ``PUSH``s. Por otro lado, ``REQ`` solo puede comunicarse con un único ``REP``. Si "``shutdown``" anula su ``REQ``, el ``REP`` con el cual compartía canal, pierde el rastro. Sucede que tales tipos de canales no unen ni 1 ni 3 nodos: solo 2. Luego, al conectar una nueva instancia a estos, la comunicación no puede evitar tener interferencias. De algún modo entonces, nuestro ``E``xpert ``A``dvisor tiene que resetearse, ya sea...
* ...de manera <u>manual</u>: nosotros haciendo clic derecho sobre la gráfica y yendo a "``Expert Advisors`` $\rightarrow$ ``Remove``"...
* ...o <u>automática</u>: "``ZMQL``" enviando un mensaje "``Shutdown``" por ``REQ``, antes de cerrar los ``Socket``s.

A modo de atajo, agregamos el argumento "``EA``" para habilitar el envío del mensaje de cierre cuando "``EA = True``". Asi que mientras tanto, dejamos el argumento en "``False``" y nos encargamos del Expert Advisor manualmente. Hasta que no armemos la función "``_send``", tendremos que recordar hacerlo manualmente siempre. La re-<u>apertura del Expert Advisor debe ser manual siempre</u>, si o si.

In [3]:
class ZMQL(ZMQL):
    def __init__(self, context, ports = ZMQL._PortsDef, verbose = 1):
        super().__init__(context, ports = ports, verbose = verbose)
    def shutdown(self, EA = True):
        #| El hilo paralelo de recepción no debe seguir funcionando al cerrar ZMQ.
        print("---------------------------------------"*2)
        for label in self.Comm.index:
            if EA and ("S" in self.Comm.loc[label, "Role"]): self._send(label, "Shutdown")
        time.sleep(0.5) #| Esperar a que responda confirmando su propia terminación.
        self.Enable["comm"] = False #| Desactivar comunicación: sockets PUSH y SUB.
        for label in self.Comm.index: #| Por cada socket...
            #| Dar el "listener" de baja, si es un socket de recepción.
            if "R" in self.Comm.loc[label, "Role"]:
                self.Poller.unregister(self.Comm["Socket"][label])
            port = self.Comm["Port"][label] #| Adquirir ID de cada socket.
            address = f"tcp://localhost:{port}" #| Armar dirección en protocolo.
            self.Comm["Socket"][label].disconnect(address) #| Disasociar de dirección.
            print(f"[[EXIT]] {label} Disconnected from port {port}.") #| Informar en consola.
        print("---------------------------------------"*2)
        # self.Thread.join() #| Unirlo con el hilo principal, y cerrar ambos ya juntos.
        del self

#### <b><u>Unit Test</u></b>

Vamos a hacer una primera prueba de lo que tenemos hasta ahora. Solo creamos una instancia, y la borramos pocos momentos despues. Todavía no creamos nuestra función "``_send``" para comunicarle a MetaTrader sobre el "``Shutdown``", <u>pero de todos modos dejamos dicho comando ya establecido</u>. Usamos entonces "``EA = False``" que ya viene por defecto para evitar problemas, y reseteamos el EA en MetaTrader manualmente para no generar interferencias con otras instancias mas adelante.

Observar que el mensaje de "Await for response" (que por cierto, es de grado "``verbose = 0``") lo dejamos formulado para dar a entender que el primer contacto con el Expert Advisor será un "handshake": una verificación de que la comunicación funciona bien. De esto nos encargaremos mas adelante, con la función "``_check``".

In [4]:
INST = ZMQL(context = ZMQ)   ;   INST.shutdown(EA = False)   ;   del INST

------------------------------------------------------------------------------
[[INIT]] SUB connecting to port 65530. Await for response...
[[INIT]] PUSH connecting to port 65531. Await for response...
[[INIT]] PULL connecting to port 65532. Await for response...
------------------------------------------------------------------------------
------------------------------------------------------------------------------
[[EXIT]] SUB Disconnected from port 65530.
[[EXIT]] PUSH Disconnected from port 65531.
[[EXIT]] PULL Disconnected from port 65532.
------------------------------------------------------------------------------


### <center><b><u>Transmisión</u></b></center>

La función "``_send``" es nuestro actuador principal. Todo comando u orden hacia el bloque vecino, será un mensaje formulado adecuadamente, enviado fundamentalmente por medio de algún ``socket`` cuyo ``Role`` sea ``S`` (``REQ`` / ``PUSH`` / ``PUB``). Proveemos entonces a su nombre ("``label``", en ``str``ing) como argumento. Lo mas adecuado sería que el mensaje contenga datos numéricos, lo mas liviano posibles, para agilizar el transporte y el posterior procesado. Pero por ahora, los formulamos como ``str``ings. Van a presentar un formato simple de 10 datos separados por punto y coma ("**``;``"**):

<h3> <center><code>message = "</code>comando<code>;</code>dato 1<code>;</code>dato 2<code>;</code>dato 3<code>;</code>dato 4<code>;</code>dato 5<code>;</code>dato 6<code>;</code>dato 7<code>;</code>dato 8<code>;</code>dato 9<code>" </code></center>

El receptor del mensaje del otro lado, va a recoger el mensaje desde su ``Socket`` de tipo ``REP`` / ``PULL`` / ``SUB`` (dependiendo el caso) y va a descifrarlo separando los 10 datos e interpretandolos a su debida manera. El "**comando**" es el primer dato de todos, y su objetivo es especificar de que tipo va a ser la solicitud. Por ejemplo: abrir una operación de trading ("``Open``"), descargar datos históricos de algún instrumento ("``OHLCV``"), etc.

Antes que nada, verificamos la correcta formulacion del mensaje. Si le faltan separadores "**``;``**", le agregamos los necesarios para llegar hasta 9. Luego, lo guardamos en el ``Cache`` del ``socket`` que utilizaremos. Luego, intentamos enviar el mensaje. Observar que para grados 1 y 2 de "``verbose``" (distintos a 0), la función nos hace un print del mensaje enviado. Si pudo enviarse, generamos una pequeña espera de 20 milisegundos con "``time.sleep``". Esto es sumamente necesario, para:
* ...darle tiempo al receptor de <u>recibirlo</u>, <u>procesarlo</u>, <u>ejecutar la acción correspondiente</u>, y <u>darnos una respuesta</u>.
* ...darle tiempo a nuestro nodo de <u>recibir la respuesta</u> y luego <u>procesarla</u>. Eso incluye guardarla en ``Cache`` también.

Puede que el sistema se haya encontrado de alguna obstrucción al enviar el mensaje. Normalmente ocurre cuando el ``Socket`` receptor del otro nodo fue borrado o se encuentra saturado. Ante tal caso, la librería "``zmq``" acciona la excepción "``zmq.error.Again``", la cual debemos usar para informar del error. Aquello es, por ejemplo, un print con grado de "``verbose = 0``". 

In [5]:
class ZMQL(ZMQL):
    def __init__(self, context, ports = ZMQL._PortsDef, verbose = 1):
        super().__init__(context, ports = ports, verbose = verbose)
    def _send(self, label, message):
        if not isinstance(label, str) or not isinstance(message, str):
            print(f"((SEND)) ERROR! Message must be a string. Got {type(message)}") ; return
        if "S" not in self.Comm.loc[label, "Role"]: return #| Solo permitir sockets de envío.
        message += ";"*(9 - message.count(";")) #| Completar con los separadores que falten.
        self.Comm.loc[label, "Cache"] = message #| Antes que nada, conservar copia del mensaje.
        try: #| Poner mensaje en "fila de espera" del socket. Enviar de inmediato apenas disponible.
            self.Comm["Socket"][label].send_string(message, zmq.DONTWAIT)
            if (self.Enable["verbose"] >= 1): print(f"<<{label}>> Command sent: [{message}]")
            time.sleep(0.02) #| Esperar un poco para darle tiempo a la llegada de la respuesta.
        except zmq.error.Again: #| Limitar la espera del mensaje. Informar cuando esperó demasiado.
            print(f"<<{label}>> Warning! Timeout with no response... try again.")

Aprovechando que ya tenemos la susodicha "``_send``", vamos a crear un primer mensaje, que deberían tener a mano todos los nodos, cualquiera sea su tarea. Esta es justamente la función "``_check``", que es la que va a ver si nuestros ``Socket``s estan bien conectados y listos para seguir.

Basicamente, consiste en:

1. Primero, enviar mensajes con la forma "``Check;;;;;;;;;``" por medio de todos los ``Socket``s que cumplan el ``Role`` de envíos ("``S``"). Se contempla que los nodos receptores tengan sus funciones para procesar este mensaje.
2. Segundo, se espera un tiempo extra de 1/4 seg. para que todos ellos hayan dado su debida respuesta.
3. Luego, se lee el ``Cache`` de cada ``Socket`` ``R``eceptor (``REP``/``PULL``/``SUB``) dentro de mi ``Comm``. Si ellos se encuentran vacíos, quiere decir que no hubo respuesta, y que por ende hubo un error de comunicación.
4. Al mas mínimo error de comunicación en cualquier ``Socket``, la función devuelve "``False``". Caso contrario, devuelve un "``True``".

In [6]:
class ZMQL(ZMQL):
    def __init__(self, context, ports = ZMQL._PortsDef, verbose = 1):
        super().__init__(context, ports = ports, verbose = verbose)
    def _check(self):
        for label in self.Comm.index: #| Enviar checks por todos los sockets de transmisión.
            if "S" in self.Comm.loc[label, "Role"]: self._send(label, "Check")
        for label in self.Comm.index: #| Revisar todos los sockets de recepción.
            time.sleep(0.25) #| Darle tiempo para recibir mensajes y llenar "Caches".
            if "R" in self.Comm.loc[label, "Role"]:
                if self.Comm["Cache"][label]: continue #| "Cache" lleno: hubo recepción
                #| Caso contrario, "Cache" vacío: no se recibió nada. Hubo alguna falla.
                print(f">>{label}<< ERROR! MQL4 \"check\" not answering!")  ;  return False
        return True #| Si está todo bien y no hubo un error en el camino, devolver "True".

<b><u>Nota</u></b>: Dentro de nuestras expresiones de ``verbose``, adoptaremos la siguiente nomenclatura:
* Si el print comienza con 2 corchetes (ej.: "``[[CHECK]]``"), implica un **aviso** de un **comando** en específico.
* Si el print comienza con 2 parentesis (ej.: "``((CHECK))``"), implica un **error** de un **comando** en específico.
* Si el print comienza con 2 flechas salientes (ej.: "``<<REQ>>``"), refiere a un ``Socket`` de tipo ``S``end.
* Si el print comienza con 2 flechas entrantes (ej.: "``>>REQ<<``"), refiere a un ``Socket`` de tipo ``R``eceive.
* Los errores terminantes llevan delante la exclamación "``ERROR!``" en mayúscula, y luego su descripción.
* Las errores no terminantes llevan delante la exclamación "``Warning!``" en minúscula, y luego su descripción.

#### <b><u>Unit test</u></b>

Es normal que el "``_check``" nos esté devolviendo una falla de comunicación en este momento. ¡Todavía no escribimos ninguna función que se encargue de revisar y procesar los mensajes recibidos! De todos modos, "``_check``" utiliza "``_send``" dentro de ella, asi que es una buena forma de:
* Probar si funciona el envío de mensajes desde el punto de vista de los ``Socket``s ``REQ`` y ``PUSH``.
* Probar si desde la perspectiva del nodo receptor, el mensaje fue recibido (o sea: ver en MetaTrader si llegó).
* Probar cuales son las limitaciones en la selección del mínimo tiempo ("``time.sleep``s") entre envío y envío.

In [7]:
INST = ZMQL(context = ZMQ)
if INST._check():
    print("[[CHECK]] Successfully initialized and connected to MetaTrader! :)")
else:
    print("((CHECK)) ERROR! Connection to MetaTrader unsuccessful. Shutting down... :(")
INST.shutdown(EA = False)   ;   del INST

------------------------------------------------------------------------------
[[INIT]] SUB connecting to port 65530. Await for response...
[[INIT]] PUSH connecting to port 65531. Await for response...
[[INIT]] PULL connecting to port 65532. Await for response...
------------------------------------------------------------------------------
<<PUSH>> Command sent: [Check;;;;;;;;;]
>>SUB<< ERROR! MQL4 "check" not answering!
((CHECK)) ERROR! Connection to MetaTrader unsuccessful. Shutting down... :(
------------------------------------------------------------------------------
[[EXIT]] SUB Disconnected from port 65530.
[[EXIT]] PUSH Disconnected from port 65531.
[[EXIT]] PULL Disconnected from port 65532.
------------------------------------------------------------------------------


Si bien "``_check``" puede convocarse manualmente en cualquier momento desde cualquier nodo, es buena idea que lo vayamos a agregar al final del "``__init__``", como parte de la inicialización del objeto "``ZMQ``". También sería necesario exigirle al sistema de no continuar con su inicialización si la comunicación no fue detectada como exitosa. Continuar trabajando con un nodo que no se puede comunicar con su entorno, no sirve, y podría traer serios problemas.

### <center><b><u>Recepción</u></b></center>

La detección, revisión y procesamiento de los mensajes recibidos, es una tarea que debería teóricamente entrar dentro de lo conocido como "<u>programación orientada a eventos</u>". En tal paradigma, se imponen ciertas condiciones en un sistema dinámico que no son constantemente revisadas, sino que son detectadas cuando se produce algún cambio en ellas.

Por ejemplo: hacer clic sobre un botón de "Aceptar" en una ventana de Windows. A primera vista, el sistema no está todo el tiempo leyendo si uno está efectivamente presionando el botón, sino que el evento de hacerlo es el que provoca la reacción. Esto parece mas eficaz que la hazaña clásica de un ciclo "``while``" infinito por si solo, que impida que el sistema pudiera estar haciendo otra cosa mientras tanto.

<u>**El ``Thread``**</u>: Hasta el momento no se encontró ninguna biblioteca de funciones orientadas a eventos dentro de Python. Generalmente es un esquema mas común en lenguajes compilados (no interpretados). Sin embargo, lo que sí tenemos aquí, son herramientas de "``threading``" que nos permiten reservar parte del kernel de Python para hacer una tarea "background" en un ``Thread`` (hilo) paralelo al principal. De ese modo, podemos incluir el susodicho ciclo "``while``" dentro de uno de ellos, sin impedir que podamos seguir interactuando con el sistema.

Pero cuidado: uno no puede interrumpir un ``Thread`` paralelo así como así, ya que es como un programa de Python funcionando por separado, independiente del principal. Si cerramos el principal, "``Thread``" podría quedar abierto. Lo que se hace es crear un "``Thread``" del tipo "``daemon``": que al cerrar el sistema, se detenga inmediatamente, cualquiera sea la situación o la tarea en la que se encuentra.

Algo <u>importante</u>: Una vez cerrada una instancia de ``ZMQL``, al ``Thread`` hay que eliminarlo para liberar espacio en el procesador de la computadora. A esto sin embargo no se lo puede hacer directamente. En cambio, se aplica la función "``join``", que lo que hace es hacer que el ``Thread`` que fue un trabajador paralelo y externo, se vuelva a incorporar a la función principal y finalice sus tareas dentro de ella. Aquella linea debe ir al final de "``shutdown``", pero es preferible dejarla fuera de estos ejemplos ya que "``shutdown``" ya fue declarada anteriormente. En todo caso, aquello queda en pendiente para la versión "``.py``".

Cuando se tienen más de un ``Thread`` funcionando en paralelo a la vez, se usa un ``str``ing "``ID``" para identificarlos. Puesto que cada objeto "``ZMQL``" tendrá un único ``Thread`` para recibir mensajes, vamos a añadir a este "``ID``" como argumento del constructor, y como identificador de cada nodo en sí.

<u>**Función "``_receive``"**</u>: será el argumento principal del ``Thread``. Es decir que todo lo que haya dentro de ella, será continuamente ejecutado en paralelo. Aquí es adonde entra en juego la variable de control "``comm``" (dentro de "``Enable``"): mientras esta sea ``True``, el ciclo "``while``" seguirá su curso. De otro modo, se detiene la lectura de mensajes recibidos, lo que funcionalmente es igual a no haber recibido nada.

Lo que primero hace "``_receive``" es estar "escuchando" a aquellos ``Socket``s incluidos dentro del "``Poller``". La función "``poll``" deja pasar a los mensajes de las filas de espera a la entrada de cada ``Socket`` de recepción. Aquellos que no hayan tenido filas o que ni siquiera sean ``R``eceptores, quedan excluidos ("``continue``").

Luego, se formula el mensaje con el método ``recv_string`` de ``Socket``. Ahora bien: en nuestro caso en particular, vamos a hacer de cuenta que <u>cualquier mensaje</u> que nos envía el Expert Advisor está <u>siempre escrito como un ``dict``</u>. Es decir:

<h3> <center><code>message = "{'</code>asunto<code>': </code>... contenido ...<code>}" </code></center>

<u>**Función "``eval``"**</u>: Resulta ser que cualquier mensaje que el interpretador de Python sea capaz de reconocer, es procesable con la función "``eval``". Es decir: si yo recibí un ``str``ing como "``'x = 3'``" o "``'{"key": "value"}'``" o cualquier cosa que pudiera parecerse perfectamente a algo escrito en Python, al aplicarle la función ``eval``, es como estar corriendolo como una línea de código más.

La consecuencia de aquello es que puedo usar a dicho mensaje como argumento de una **nueva función**, pero <u>no como ``str``ing, sino como el mismisimo ``dict`` que implica ser</u>. Esta nueva función será llamada "``_process``". Tendrá todo el contenido necesario para procesar el ``dict`` de la manera que tenga que hacerlo cada nodo de la red ZMQ según a que **subclase de ZMQL** pertenezca (o sea; que tarea tenga que hacer). 

Escribiremos una "``_process``" genérica como prueba, mas adelante. Eso si: debe ser especificado de antemano como ``arg``umento dentro de la "``Thread``". Lo bueno de esto, es que entonces "``_receive``" se termina convirtiendo en una suerte de plantilla o placeholder, a modo de: "*«insertar de que modo quiere usar los mensajes ya ``eval``uados, aquí»*".

Por otro lado, puede suceder que el mensaje no esté escrito como ninguna línea de Python ``eval``uable. Esto sería un problema de MQL4 que mandó el mensaje, no de Python en sí. Como "``verbose = 0``" solo reporta errores funcionales a este nodo, se deja entonces como un ``warning`` ``print``eable en grado mayor (``1`` y ``2``).

Finalmente, luego de que el mensaje es guardado dentro del ``Cache`` del ``Socket`` receptor, se lo exhibe en consola dependiendo del ``Socket`` del cual provino. Recordar que:
* Para mensajes **PUSH** - **PULL**, los cuales son eventuales y esporádicos, alcanza con un "``verbose = 1``".
* Para mensajes **PUB** - **SUB**, los cuales son continuos e ininterrumpidos, se debe usar "``verbose = 2``".
<br><u>Pero cuidado</u>: todo lo que se imprime en un ``Thread`` paralelo al principal, no se muestra en un "``.ipynb``".
<br>Solo podrán verse los ``print``s dentro de  "``_receive``" cuando ésta corre en un "``.py``" desde una <u>terminal</u>.

<b><u>"``Base``" de datos</u></b>: será el repositorio adonde manipulemos y guardemos toda la información que necesitemos usar para nuestro sistema de trading en las subclases futuras. Tendrá formato "``dict``", y su contenido será especificado y organizado especialmente según la tarea principal que tenga que desempeñar. Pero en general, cada función "``_process``" amoldará los mensajes y los almacenará de algún modo en "``Base``". Por ahora, la dejaremos vacía.

In [8]:
class ZMQL(ZMQL):
    def __init__(self, context, ID = "Test", ports = ZMQL._PortsDef, verbose = 1):
        super().__init__(context, ports = ports, verbose = verbose)
        self.Base = pandas.Series() #| Comunmente "dict". En este caso, usamos "series".
        self.Thread = threading.Thread(name = ID, target = self._receive, daemon = True)
        self.Thread.start()
    def _receive(self):
        while self.Enable["comm"]: #| Dentro del hilo paralelo, siempre y cuando este parámetro de control sea "True"...
            sockets_polled = dict(self.Poller.poll()) #| Chequear cuales son los puertos que han recibido algo.
            for label in self.Comm.index: #| Tomar a cada uno de los puertos que tengo registrados.
                Socket = self.Comm["Socket"][label] #| De ellos, tomar a cada uno de los sockets.
                if (Socket not in sockets_polled.keys()): continue #| Saltear los que no son de recepción. 
                if (sockets_polled[Socket] != zmq.POLLIN): continue #| Saltear los que no hayan recibido nada.
                try: message = self.Comm["Socket"][label].recv_string(zmq.DONTWAIT) #| Formular respuesta como string.
                except: message = None #| Si no se pudo formular el string de respuesta, es que no hubo respuesta.
                if message: #| Si se formuló una respuesta, parsearla literalmente como una "linea de Python".
                    try: self._process(eval(message)) #| Por ejemplo: "eval('x = 2')" hace que "x" sea "2".
                    #| "_process" va a ser una función que va a procesar la variable implicada.
                    except AssertionError: #| Dada una condición de cierre explicita dentro de "_process"...
                        print(f">>{label}<< ERROR! Forced shutdown condition --> {message}")
                        self.Enable["comm"] = False #| Detención del proceso...
                    except Exception as ex: #| Cuando hubo un error al procesarla...
                        if (self.Enable["verbose"] >= 1): #| Si se activó el verbose de errores "no graves"...
                            Type = str(type(ex))[8:-2] #| Conseguir el tipo de error en formato string.
                            Line = ex.__traceback__.tb_next.tb_next.tb_lineno #| Conseguir nº de linea del error.
                            warning = f"{Type}: {ex.args[0]}..." #| Armar mensaje de error y mostrar.
                            print(f">>{label}<< Warning! Process error at line {Line} ===>", warning)
                        continue #| No bloquear el bucle paralelo de recepción luego de un error de recepción.
                    self.Comm.loc[label, "Cache"] = message #| Guardar string en Cache de puerto tal y como llegó.
                    v = self.Enable["verbose"] #| 1º grado incluye todos los "R" menos "SUB". 2º grado incluye "SUB". 
                    if (v > 1) or (v and (label != "SUB")): print(f">>{label}<< Received -> {message}", flush = True)
                    sys.stdout.flush() #| Mostrar cualquier cosa que haya sido impresa por esta vía, en consola.
                    time.sleep(0.001) #| Apenas pausar al sistema para impedir saturación de punto de recepción.

#### <b><u>Función "``_process``"</u></b>

Mas adelante, cada subclase dentro de la red tendrá su propia "``_process``", que lisa y llanamente dictaminará "que hacer" con el ``dict`` que el mensaje transportaba. Ya sea hacer cálculos con su contenido, actualizar una base de datos, o simplemente guardarlo en algún lugar, todo aquello debe estar contemplado en esta función. También, como está ubicada dentro de un bloque "``try`` - ``except``" en la celda anterior, puede contener excepciones del tipo "``AssertionError``" cuando haya algo en el mensaje que requiriese detener la rutina de comunicación.

Como estamos en un notebook y no podemos ver los mensajes que van llegando por "``SUB``", vamos a usar y ``print``ear el "``dict``" como ``Base``"  de datos. Solo haremos esto aquí como prueba, no en el futuro "``.py``". Así, ya sean respuestas de futuros "``_check``", "``_shutdown``" o de cualquiera de los comandos ya redactados en el Expert Advisor de MetaTrader, todas van a aparecer acá, con su fecha y hora ("``datetime.datetime.now``") como ``index``.

Algo <u>sumamente importante</u> es que, como ya tenemos nuestro sistema de recepción de mensajes listo, podemos ya adicionar la ya explicada función "``_check``" y su condición de continuidad, dentro del constructor "``__init__``".

In [9]:
from random import random
class ZMQL(ZMQL):
    def __init__(self, context, ID = "Test", ports = ZMQL._PortsDef, verbose = 1):
        assert isinstance(ID, str), "((INIT)) ERROR! \"ID\" string must differ from other block IDs."
        super().__init__(context, ID = ID, ports = ports, verbose = verbose)
        if self._check(): print("[[CHECK]] Successfully initialized and connected to MetaTrader! :)")
        else: print("((CHECK)) ERROR! Connection unsuccessful. Shutting down... :(", self.shutdown())
    def _process(self, message):
        assert not (len(self.Base) > 100)  #| Si supera las 100 filas, detener.
        self.Base[datetime.datetime.now()] = str(message)  #| Añadir al final de "log".
        if (random() < 0.05): return  #| 1 de cada 20 veces, no pasará a la próxima linea.
        self.Base = self.Base[-100:]  #| Limitar Base a 100 filas.

#### <u><b>Unit Test</b></u>

Observar en la celda anterior que limitamos el largo de este log a 100 filas. Pero vamos a hacer una prueba: de manera intencional, le damos una pequeña probabilidad (de 5%) de que se exceda, haciendo uso de la función "``random``". Si sucede esto, vamos a convocar la excepción "``AssertionError``" para simular una condición crítica y detener el proceso de comunicación.

Adicionalmente, probaremos la función "``_check``" ahora dentro del "``__init__``". Para hacer funcionar la siguiente celda, debemos verificar que MQL4 se encuentra ya funcionando y listo para responder. De otro modo, nos devolverá un error como la vez anterior. Se recomienda probar distintos grados de "``verbose``" para poder ver que es lo que deja reportar cada uno.

En la siguiente celda, comentar y descomentar las lineas de la **(1)** a la **(7)** según corresponda...
* Para hacer una prueba de conexión rápida y completa, descomentar y correr todas.
<br>Leer la consola: si no se muestran errores, la comunicación funciona a la perfección.
* Para crear una instancia actualizada de ``ZMQL``, en principio descomentar solamente **(1)** y **(2)**.
<br>Excepto "``Ticks``": si se desea, cambiar contenido del "``_send``" (``; symbol ; frame ; slot``).
* Para empezar a ver los mensajes (de ``Ticks``) que llegaron a la ``Base``, descomentar solo **(3)** y **(4)**.
<br>Ejecutar reiterativamente, vigilando la cantidad de mensajes. Prestar atención cuando llegan a 100.
* En caso de que sean 101, es porque el "``AssertionError``" se activó, y "``comm``" se volvió "``False``".
<br> Notar que MQL sigue mandando ``Ticks``, pero ``ZMQL`` no los guarda. Descomentar **(5)** para reanudar.
* Para desuscribirse del ``symbol`` escogido y detener el envío de ``Ticks``, descomentar y correr solo **(6)**.
<br>La "``frame``" debe ser sí o sí igual a cero, y el "``slot``" debe ser igual al utilizado antes en la linea **(2)**.
* Para finalizar el proceso y eliminar la instancia de "``ZMQL``" y sus sockets, ejecutar solamente la linea **(7)**.

**Verificar que MetaTrader esté abierto y con su Expert Advisor corriendo en pantalla!!**

In [21]:
INST = ZMQL(context = ZMQ)                                   #| (1) Crear instancia lista para recibir mensajes.
INST._send(list(ZMQL._PortsDef)[1], "Ticks;BTCUSD;1;0")      #| (2) Enviar suscripción a BTCUSD S1 en slot 0.
print("Comm =>", INST.Enable["comm"], "| Log end:")          #| (3) Prestar atención a estado de "comm".
print(INST.Base[-10:], "\nLines:", len(INST.Base))           #| (4) Mostrar Log, y cantidad de lineas guardadas. 
INST.Enable["comm"] = True                                   #| (5) Habilitar lectura de mensajes recibidos.
INST._send(list(ZMQL._PortsDef)[1], "Ticks;BTCUSD;0;0")      #| (6) Desuscripción BTCUSD. Notar cambios en MetaTrader.
INST.shutdown(False)   ;   del INST                          #| (7) Desconectar instancia de ZMQ y eliminarla.

------------------------------------------------------------------------------
[[INIT]] SUB connecting to port 65530. Await for response...
[[INIT]] PUSH connecting to port 65531. Await for response...
[[INIT]] PULL connecting to port 65532. Await for response...
------------------------------------------------------------------------------
<<PUSH>> Command sent: [Check;;;;;;;;;]
>>PULL<< Received -> {'Check': 'PUSH'}
[[CHECK]] Successfully initialized and connected to MetaTrader! :)
<<PUSH>> Command sent: [Ticks;BTCUSD;1;0;;;;;;]
>>PULL<< Received -> {'Ticks': ['BTCUSD', 1.0]}
Comm => True | Log end:
2020-10-29 15:25:35.751848                                     {'Check': 'SUB'}
2020-10-29 15:25:35.757831                                    {'Check': 'PUSH'}
2020-10-29 15:25:36.518663                           {'Ticks': ['BTCUSD', 1.0]}
2020-10-29 15:25:36.532627    {'BTCUSD': [1604006714.0, 13579.87, 13583.87, ...
dtype: object 
Lines: 4
<<PUSH>> Command sent: [Ticks;BTCUSD;0;0;;;;;;]